切資料


In [ ]:
import os
import numpy as np
import csv

def slope(input_data):
    slope_col = np.zeros(len(input_data),dtype=float)
    for i in range(len(input_data) - 1):
        for j in range(1,4):
            temp = input_data[i+1][j] - input_data[i][j]
            slope_col[i] += temp**2
        slope_col[i] = slope_col[i]**0.5
    return slope_col

def ma(slope_col, n):
    slope_ma = np.zeros(len(slope_col),dtype=float)
    for i in range(n,len(slope_col)-n):
        for j in range(-n,n+1):
            slope_ma[i] += slope_col[i+j]
        slope_ma[i] /= float(2*n+1)
    return slope_ma

def data_cut(input_data, save_data, n, ball_type):
    slope_col = slope(input_data)
    slope_avg = np.average(slope_col)
    slope_ma = ma(slope_col, n)

    for i in range(60,len(slope_col)-50):

        if ( slope_ma[i] > slope_avg ) and ( slope_col[i]==max(slope_col[i-50:i+50]) ) : # 找到可能峰值
            start = 0  # 向前&向後找起點
            end = 0
            while i+start > (50+n) :
                start -= 1
                if slope_ma[i+start] <= slope_avg:
                    break
            while i+end < (len(slope_col)-50-n) :
                end += 1
                if slope_ma[i+end] <= slope_avg:
                    break
            if end-start > 50: # 依球種分類存入 save_data
                if ball_type == "high": # 長遠球
                    save_data.append(input_data[i-60:i+40, [1,2,3,5,6,7]])
                elif ball_type == "short": # 挑球
                    save_data.append(input_data[i-40:i+20, [1,2,3,5,6,7]])

# Removes null bytes from the input file and returns a sanitized version of the file.
def sanitize_file(input_file_path):
    sanitized_content = ""
    with open(input_file_path, 'r', encoding='utf-8', errors='replace') as f:
        content = f.read()
        sanitized_content = content.replace('\x00', '')

    # Overwrite the original file with sanitized content
    with open(input_file_path, 'w', encoding='utf-8') as f:
        f.write(sanitized_content)


# 指定資料集所在的路徑
data_path = "/content/"

# 取得該路徑下所有的檔案名稱
all_files = os.listdir(data_path)

# 過濾出所有的 .txt 檔案
txt_files = [file_name for file_name in all_files if file_name.endswith('.txt')]

for t, input_filename in enumerate(txt_files):

    input_file_path = os.path.join(data_path, input_filename)

    # Sanitize the file by removing null bytes
    sanitize_file(input_file_path)

    # Read the entire file into memory
    with open(input_file_path, 'r', encoding='utf-8', errors='replace') as f:
        lines = f.readlines()

    # Starting from the last line, move upwards until a complete line (with 7 commas) is found
    while lines and lines[-1].count(",") != 7:
        lines = lines[:-1]

    # Load the (potentially modified) data into a numpy array
    input_data = np.loadtxt(lines, delimiter=",", dtype=float)
    # print(input_filename)
    # print(input_data)

    # input_data = np.loadtxt(input_file_path, delimiter=",", dtype=float) # 從文本文件中讀取數據，以","為分隔符號

    save_data = []
    n = 3   # n:取移動平均用，看前後需要取幾個

    #決定球種
    if "high" in input_filename.lower():
        ball_type = "high"
    elif "short" in input_filename.lower():
        ball_type = "short"
    else:
        print("error txt-file")

    data_cut(input_data, save_data, n+1, ball_type)

    save_data = np.array(save_data).astype(float)

    print(input_filename)
    print(len(save_data))

    output_filename = input_filename.split(".")[0] + "_cut.csv"

    output_file_path = data_path + "\\" + output_filename

    with open(output_file_path, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=',')
        for data in save_data:
            formatted_data = [','.join(str(val) for val in row) for row in data]
            writer.writerow(formatted_data)

Data to Array

In [ ]:
import os
import numpy as np
import csv

#常數放置區
hittype = 1       #長球:0, 短球:1
both_cols = 6
high_rows = 100
short_rows = 60
high_file_end = 'high_cut.csv'
short_file_end = 'short_cut.csv'
#常數放置區
#-----csv_cutdata to train_test_data----------
# 指定数据集所在的路径
data_path = '.'

# 取得该路径下所有的文件名
all_files = os.listdir(data_path)

#長球:high_cut.csv, 短球:short_cut.csv
dynamic_content = "error"
if(hittype == 0):
  dynamic_content = high_file_end
elif(hittype == 1):
  dynamic_content = short_file_end
#print(dynamic_content)

# 過濾出指定的 .csv 檔案
target_files = [file_name for file_name in all_files if file_name.endswith(dynamic_content)]
print(target_files)
# 讀取檔案至三維矩陣
num_rows = 0
if(hittype == 0):
  num_rows = high_rows
elif(hittype == 1):
  num_rows = short_rows
num_cols = both_cols

data = np.empty((0, num_rows, num_cols), dtype=int)
labels = np.empty((0, 5), dtype=int)
for i,file_name in enumerate(target_files):
  file_path = os.path.join(data_path, file_name)
  with open(file_path, 'r', newline='') as f:
        read = csv.reader(f, delimiter=',')
        read_list = list(read)
        #print(len(read_list))
  #print(len(read_list))
  for j in range(len(read_list)):
    #print(len(read_list[0]))
    for k in range(len(read_list[0])):
      read_list[j][k] = list(map(float, read_list[j][k].split(',')))
    # Add the new column to the labels array
    new_row = np.array([file_name[7], file_name[8], file_name[9], file_name[10], file_name[11]], dtype=int)
    new_row = new_row - 1
    labels = np.vstack((labels, new_row))
  data = np.concatenate((data, read_list))

numLable = len(target_files)
#data=data.reshape(len(labels),num_rows*num_cols)
print(data.shape)
print(labels.shape)
print(len(target_files))
print(labels)

['\\s16_n_22122_109206014_short_cut.csv', '\\s10_y_44444_109703003_short_cut.csv', '\\s03_y_33222_110358030_short_cut.csv', '\\s07_y_11221_110703008_short_cut.csv', '\\s11_n_22222_110308047_short_cut.csv', '\\s12_y_22121_109703018_short_cut.csv', '\\s09_y_33222_109703056_short_cut.csv', '\\s08_n_22232_110302059_short_cut.csv', '\\s01_y_13231_234234234_short_cut.csv', '\\s14_y_44443_109507014_short_cut.csv', '\\s04_y_11121_101108044_short_cut.csv', '\\s06_n_12121_111206016_short_cut.csv', '\\s05_y_44334_109703024_short_cut.csv', '\\s13_n_11111_109540020_short_cut.csv', '\\s15_y_11111_109305061_short_cut.csv', '\\s02_y_11111_123123123_short_cut.csv']
(230, 60, 6)
(230, 5)
16
[[1 1 0 1 1]
 [1 1 0 1 1]
 [1 1 0 1 1]
 ...
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]


In [ ]:
from sklearn.model_selection import train_test_split
#定義訓練集和測試集的比例，例如訓練集佔 80%，測試集佔 20%
train_ratio = 0.8
test_ratio = 0.2
#labels = labels2
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=test_ratio, random_state=24)

In [ ]:
print(x_train.shape)
print(x_test.shape)
x_train = x_train.reshape(184, 60, 6)
x_test = x_test.reshape(46, 60, 6)

(184, 60, 6)
(46, 60, 6)


In [ ]:
!pip install tensorflow
!pip install tflite_runtime

In [ ]:
import tensorflow as tf
import numpy as np
import tflite_runtime.interpreter as tflite
import os
import csv
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
class MyDataset:
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        x = self.data[index]
        y = self.labels[index]
        return x, y


In [ ]:
#常數放置區
numEpoch = 30
batch_size = 8

if(hittype == 0):
  Time_Step = 100
elif(hittype == 1):
  Time_Step = 60

Input_Size = 6
learning_rate = 0.01

Hidden_Size = 16
Num_Layers = 2
# Output_Size由類別數決定，若分n類，則為n
# 這裡 = len(target_files)是因為同一人的資料不會出現第2次
# 未來 = 錯誤類別數
Output_Size = 5

number_of_samples = 3
momentum = 0.9

#定義訓練集和測試集的比例，例如訓練集佔 80%，測試集佔 20%
train_ratio = 0.8
test_ratio = 0.2

In [ ]:
output_dim = 5

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=test_ratio, random_state=42)
#x_train, y_train, x_test, y_test = map(torch.tensor, (x_train, y_train, x_test, y_test))

In [ ]:
print(x_train.shape, x_test.shape)
print(y_train.shape, y_test.shape)
X_train = x_train.reshape(184, 360)
X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test = x_test.reshape(46, 360)
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])
print(X_train.shape, X_test.shape)

(184, 60, 6) (46, 60, 6)
(184, 5) (46, 5)
(184, 1, 360) (46, 1, 360)


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Dense, Input

# Define custom output function
def custom_output(x):
    # Implement your custom output logic here
    return x

# Define custom loss function
import tensorflow as tf

def custom_loss(y_true, y_pred):
    y_true = tf.cast(y_true, dtype=tf.float32)  # Cast y_true to float32
    loss = tf.reduce_mean(tf.square(y_true - y_pred))
    return loss

# Define custom accuracy function
threshold = 0.5
def custom_accuracy(y_true, y_pred):
    abs_diff = tf.abs(y_true - y_pred)
    condition = tf.less_equal(abs_diff, threshold)
    acc = tf.cast(condition, tf.float32)
    return acc

# Create a custom LSTM model
class CustomLSTMModel(Model):
    def __init__(self, units, output_dim):
        super(CustomLSTMModel, self).__init__()
        self.lstm = LSTM(units)
        self.dense = Dense(output_dim, activation=custom_output)

    def call(self, inputs):
        # x = self.lstm(inputs)
        # return self.dense(x)
        lstm_output = self.lstm(inputs)
        output = self.dense(lstm_output)
        output = custom_output(output)
        return output

# Instantiate the custom model
model = CustomLSTMModel(units=64, output_dim=output_dim)

# Compile the model with custom loss and accuracy functions
model.compile(optimizer='adam', loss=custom_loss, metrics=[custom_accuracy])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss}, Custom Accuracy: {accuracy}')


Epoch 1/10
6/6 [==============================] - 2s 7ms/step - loss: 1.1926 - custom_accuracy: 0.4522
Epoch 2/10
6/6 [==============================] - 0s 7ms/step - loss: 0.7343 - custom_accuracy: 0.5359
Epoch 3/10
6/6 [==============================] - 0s 6ms/step - loss: 0.5883 - custom_accuracy: 0.6000
Epoch 4/10
6/6 [==============================] - 0s 6ms/step - loss: 0.4930 - custom_accuracy: 0.6641
Epoch 5/10
6/6 [==============================] - 0s 6ms/step - loss: 0.4187 - custom_accuracy: 0.7130
Epoch 6/10
6/6 [==============================] - 0s 7ms/step - loss: 0.3574 - custom_accuracy: 0.7696
Epoch 7/10
6/6 [==============================] - 0s 10ms/step - loss: 0.3084 - custom_accuracy: 0.8054
Epoch 8/10
6/6 [==============================] - 0s 7ms/step - loss: 0.2670 - custom_accuracy: 0.8402
Epoch 9/10
6/6 [==============================] - 0s 7ms/step - loss: 0.2309 - custom_accuracy: 0.8707
Epoch 10/10
2/2 [==============================] - 1s 10ms/step - loss: 

In [ ]:
# Evaluate the model
y_pred = model.predict(X_test)
custom_accuracies = custom_accuracy(y_test, y_pred)

for i in range(5):
    accuracy_i = custom_accuracies[:, i]
    accuracy_i_value = tf.reduce_mean(accuracy_i).numpy()
    print(f'Custom Accuracy Component {i + 1}: {accuracy_i_value}')


2/2 [==============================] - 0s 12ms/step
Custom Accuracy Component 1: 0.6521739363670349
Custom Accuracy Component 2: 0.695652186870575
Custom Accuracy Component 3: 0.8478260636329651
Custom Accuracy Component 4: 0.782608687877655
Custom Accuracy Component 5: 0.8913043737411499


In [ ]:
import os
import numpy as np
import csv

#常數放置區
hittype = 1       #長球:0, 短球:1
both_cols = 6
high_rows = 100
short_rows = 60
high_file_end = 'high_cut.csv'
short_file_end = 'short_cut.csv'
#常數放置區
#-----csv_cutdata to train_test_data----------
# 指定数据集所在的路径
data_path = '.'

# 取得该路径下所有的文件名
all_files = os.listdir(data_path)

#長球:high_cut.csv, 短球:short_cut.csv
dynamic_content = "error"
if(hittype == 0):
  dynamic_content = high_file_end
elif(hittype == 1):
  dynamic_content = short_file_end
#print(dynamic_content)

# 過濾出指定的 .csv 檔案
target_files = '\s04_y_11121_101108044_short_cut.csv'
print(target_files)
# 讀取檔案至三維矩陣
num_rows = 0
if(hittype == 0):
  num_rows = high_rows
elif(hittype == 1):
  num_rows = short_rows
num_cols = both_cols
file_name = '\s03_y_33222_110358030_short_cut.csv'
the_scored_file = np.empty((0, num_rows, num_cols), dtype=int)
the_scored = np.empty((0, 5), dtype=int)
file_path = os.path.join(data_path, file_name)
with open(file_path, 'r', newline='') as f:
      read = csv.reader(f, delimiter=',')
      read_list = list(read)
      #print(len(read_list))
#print(len(read_list))
for j in range(len(read_list)):
  #print(len(read_list[0]))
  for k in range(len(read_list[0])):
    read_list[j][k] = list(map(float, read_list[j][k].split(',')))
  # Add the new column to the labels array
  new_row = np.array([file_name[7], file_name[8], file_name[9], file_name[10], file_name[11]], dtype=int)
  new_row = new_row - 1
  the_scored = np.vstack((the_scored, new_row))
the_scored_file = np.concatenate((the_scored_file, read_list))

numLable = len(target_files)
print(the_scored_file.shape)
the_scored_file = the_scored_file.reshape(the_scored_file.shape[0], 360)
the_scored_file = the_scored_file.reshape(the_scored_file.shape[0], 1, the_scored_file.shape[1])

\s04_y_11121_101108044_short_cut.csv
(7, 60, 6)


In [ ]:
import numpy as np

def custom_prediction(swings):
    processed_swings = np.array(swings)
    scores = model.predict(processed_swings)
    #average_scores = np.mean(scores, axis=0)
    average_scores = np.mean(np.round(scores), axis=0)
    rounded_average_scores = np.round(average_scores)+1
    return rounded_average_scores

# Obtain the average scores for the actions
average_scores = custom_prediction(the_scored_file)
print("Average Scores (Rounded):", average_scores)


1/1 [==============================] - 0s 23ms/step
Average Scores (Rounded): [2. 3. 2. 2. 1.]
